In [1]:
import pyautogui
import numpy as np
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import matplotlib.pyplot as plt
import math
import cv2
import collections

/home/jnu/anaconda3/envs/jupyter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# DQN 모델 정의
class DQN(nn.Module):
    def __init__(self,num_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 64, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(64, 256, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(nn.Conv2d(256, 512, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(inplace=True))
        self.actor_linear = nn.Sequential(nn.Linear(933888, 256),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(256,num_actions))

    def forward(self, x): #각 action에 대한 가치출력
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.actor_linear(x.reshape(x.size(0), -1))
        return x
    
    def sample_action(self,obs,epsilon):
        out=self.forward(obs)
        coin=random.random()
        if coin<epsilon:
            return random.randint(0,509)
        else:
            return out.argmax().item()

In [ ]:
def __init__(self,num_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 64, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(64, 256, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(nn.Conv2d(256, 512, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(inplace=True))
        self.actor_linear = nn.Sequential(nn.Linear(933888, 256),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(256,num_actions))
def __init__(self,num_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 64, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(64, 256, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(nn.Conv2d(256, 512, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(inplace=True))
        self.actor_linear = nn.Sequential(nn.Linear(933888, 13))

        

In [3]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            state, action, reward, s_prime, done_mask = transition
            s_lst.append(state)
            a_lst.append([action])
            r_lst.append([reward])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [4]:
x_list=[]
y_list=[]
for i in range(14):
    y_list.append(508+(19*i))
    y_list.append(202+(19*i))
    
for i in range(18):
    x_list.append(791+(24*i))
    action_list=[(x,y) for x in x_list for y in y_list]
    action_list.append((900,900)) #1번 카드(252)
    action_list.append((1000,900)) #2번 카드(253)
    action_list.append((1100,900)) #3번 카드(254)
    action_list.append((1200,900)) #4번 카드(255)
    action_list.append((858,478)) #왼쪽 다리
    action_list.append((1125,478)) #오른쪽 다리
    num_actions=len(action_list)

action_set={}
for i in range(num_actions):
    action_set[i]=action_list[i]

In [5]:
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 24

In [6]:
#게임 환경 클래스(state, reward, done 이 들어가야함)
class ENV:
    def __init__(self):
        self.done=0
        self.reward=0
        
    def get_state(self):
        screenshot=pyautogui.screenshot(region=(741,100,1250-741,999-100))
        screenshot.save('screenshot.png')
        state=np.array(screenshot)
        state=np.float32(state)/255.0
        state=torch.from_numpy(state).unsqueeze(0).float()
        state=state.permute(0,3,1,2)
        return state # state는 tensor이고 (1,3,509,899)

    def get_reward(self):
        screenshot=cv2.imread('screenshot.png')
        lose=cv2.imread('lose.png')
        win=cv2.imread('win.png')
        score_lose=cv2.matchTemplate(screenshot,lose,cv2.TM_CCOEFF_NORMED)
        score_win=cv2.matchTemplate(screenshot,win,cv2.TM_CCOEFF_NORMED)
        reward=0
        done=0 #게임 한 번
        if score_lose[0][0]>=0.9:
            reward-=3000000
            done=1
        
        #win 사진
        elif score_win[0][0]>=0.9:
            reward+=3000000
            done=1
            
        return reward, done
    
    def get_reward2(self):
        enemy_l=pyautogui.screenshot(region=(825,223,900-825,243-223))
        enemy_r=pyautogui.screenshot(region=(1090,223,1165-1090,243-223))
        enemy_m=pyautogui.screenshot(region=(942,114,1052-942,145-114))
        my_l=pyautogui.screenshot(region=(825,648,900-825,668-648))
        my_r=pyautogui.screenshot(region=(1090,648,1165-1090,668-648))
        my_m=pyautogui.screenshot(region=(942,767,1052-942,798-767))

        return my_l,my_r,my_m,enemy_l,enemy_r,enemy_m
    
    #메뉴칸 (1223,189) #훈련캠프 (1068,384) #확인 (1082,609) #끝나고 확인버튼 (990,862)
    def replay(self):
        time.sleep(1)
        pyautogui.moveTo(990,862)
        pyautogui.click(button='left')
        time.sleep(1)
        pyautogui.moveTo(1223,189)
        pyautogui.click(button='left')
        time.sleep(1)
        pyautogui.moveTo(1068,384)
        pyautogui.click(button='left')
        time.sleep(1)
        pyautogui.moveTo(1082,609)
        pyautogui.click(button='left')
        
    
    #def select_action(self):
        

In [7]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
def diff(image1,image2):
    score=cv2.matchTemplate(image1,image2,cv2.TM_CCOEFF_NORMED)
    return score[0][0]

In [9]:
def main():
    env = ENV()
    q = DQN(num_actions)
    q_target = DQN(num_actions)
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(40):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        state = env.get_state()
        reward,done=env.get_reward()[0],env.get_reward()[1]

        while done==0:
            '''
            my_l,my_r,my_m,enemy_l,enemy_r,enemy_m=env.get_reward2()[0],env.get_reward2()[1],env.get_reward2()[2],env.get_reward2()[3],env.get_reward2()[4],env.get_reward2()[5]
            my_l.save('my_l.png')
            my_l=cv2.imread('my_l.png')
            my_r.save('my_r.png')
            my_r=cv2.imread('my_r.png')
            my_m.save('my_m.png')
            my_m=cv2.imread('my_m.png')
            enemy_l.save('enemy_l.png')
            enemy_l=cv2.imread('enenmy_l.png')
            enemy_r.save('enemy_r.png')
            enemy_r=cv2.imread('enenmy_r.png')
            enemy_m.save('enemy_m.png')
            enemy_m=cv2.imread('enenmy_m.png')
            '''
            action_= q.sample_action(state, epsilon)
            action=action_set[action_]
            
            pyautogui.moveTo(action[0],action[1])
            pyautogui.click(button='left')
            '''
            s_prime, reward, done = env.get_state(),env.get_reward()[0],env.get_reward()[1]
            new_my_l,new_my_r,new_my_m,new_enemy_l,new_enemy_r,new_enemy_m=env.get_reward2()[0],env.get_reward2()[1],env.get_reward2()[2],env.get_reward2()[3],env.get_reward2()[4],env.get_reward2()[5]
            new_my_l.save('new_my_l.png')
            new_my_l=cv2.imread('new_my_l.png')
            new_my_r.save('new_my_r.png')
            new_my_r=cv2.imread('new_my_r.png')
            new_my_m.save('new_my_m.png')
            new_my_m=cv2.imread('new_my_m.png')
            new_enemy_l.save('new_enemy_l.png')
            new_enemy_l=cv2.imread('new_enemy_l.png')
            new_enemy_r.save('new_enemy_r.png')
            new_enemy_r=cv2.imread('new_enemy_r.png')
            new_enemy_m.save('new_enemy_m.png')
            new_enemy_m=cv2.imread('new_enemy_m.png')
        
            if diff(my_l,new_my_l)==1.0:
                reward=reward
            else:
                reward-=100
            
            if diff(my_r,new_my_r)==1.0:
                reward=reward
            else:
                reward-=100
            
            if diff(my_m,new_my_m)==1.0:
                reward=reward
            else:
                reward-=100
            '''    
            
            done_mask = 0.0 if done else 1.0
            memory.put((state,action,reward/100.0,s_prime, done_mask))
            state = s_prime

            score += reward
            
            if done==1:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        
        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0

if __name__ == '__main__':
    main()

[ WARN:0@9.412] global loadsave.cpp:244 findDecoder imread_('enenmy_l.png'): can't open/read file: check file path/integrity
[ WARN:0@9.413] global loadsave.cpp:244 findDecoder imread_('enenmy_r.png'): can't open/read file: check file path/integrity
[ WARN:0@9.413] global loadsave.cpp:244 findDecoder imread_('enenmy_m.png'): can't open/read file: check file path/integrity
[ WARN:0@23.501] global loadsave.cpp:244 findDecoder imread_('enenmy_l.png'): can't open/read file: check file path/integrity
[ WARN:0@23.502] global loadsave.cpp:244 findDecoder imread_('enenmy_r.png'): can't open/read file: check file path/integrity
[ WARN:0@23.502] global loadsave.cpp:244 findDecoder imread_('enenmy_m.png'): can't open/read file: check file path/integrity


n_episode :20, score : -150015.0, n_buffer : 2, eps : 7.9%


In [10]:
print(f'loss mean : {np.mean(losses[:-1])}')
steps = np.arange(1, step)
plt.plot(steps, losses[:-1], 'r', label='Training loss')
plt.title('Training loss')
plt.xlabel('Click')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: name 'losses' is not defined